In [1]:
import sys
sys.path.append("..")

In [2]:
from ADGAT.Model import *
from ADGAT.utils import *
import pickle
import torch
from torch import optim
from tqdm import tqdm

In [3]:
# args:
device = 0
task = 1
max_epoch = 300
wait_epoch = 30
eta = 1e-4
lr = 5e-4
heads_att = 6
hidn_att = 60
hidn_rnn = 360
weight_constraint = 0 # L2 weight constraint
rnn_length = 30
dropout = 0.2
clip = 0.25
infer = 1
relation = 'supply'
save = True 

In [4]:
def load_dataset(DEVICE, relation):
    with open('../Data/relations_author_source/x_numerical.pkl', 'rb') as handle:
        markets = pickle.load(handle)
    with open('../Data/relations_author_source/y_.pkl', 'rb') as handle:
        y_load = pickle.load(handle)

    markets = markets.astype(np.float64)
    x_market = torch.tensor(markets, device=DEVICE)
    x_market.to(torch.double)

    #
    alternatives = np.zeros(list(markets.shape[:-1]) + [1]).astype(np.float64)
    x_alternative = torch.tensor(alternatives, device=DEVICE)
    x_alternative.to(torch.double)
    
    if relation != "None":
        with open('../Data/relations_author_source/' + relation + '_relation.pkl', 'rb') as handle:
            relation_static = pickle.load(handle)
        relation_static = torch.tensor(relation_static, device=DEVICE)
        relation_static.to(torch.double)
    else:
        relation_static = None
    y = torch.tensor(y_load, device=DEVICE)
    y = (y>0).to(torch.long)


    return x_market, y, x_alternative, relation_static

def train(model, x_train, x_alt_train, y_train, relation_static = None):
    model.train()
    seq_len = len(x_train)
    train_seq = list(range(seq_len))[rnn_length:]
    random.shuffle(train_seq)
    total_loss = 0
    total_loss_count = 0
    batch_train = 15
    for i in tqdm(train_seq):
        output = model(x_train[i - rnn_length + 1: i + 1], x_alt_train[i - rnn_length + 1: i + 1],  relation_static = relation_static)
        loss = criterion(output, y_train[i])
        loss.backward()
        total_loss += loss.item()
        total_loss_count += 1
        if total_loss_count % batch_train == batch_train - 1:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            optimizer.zero_grad()
    if total_loss_count % batch_train != batch_train - 1:
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
    return total_loss / total_loss_count

def evaluate(model, x_eval, x_alt_eval, y_eval, relation_static = None):
    model.eval()
    seq_len = len(x_eval)
    seq = list(range(seq_len))[rnn_length:]
    preds = []
    trues = []
    for i in seq:
        output = model(x_eval[i - rnn_length + 1: i + 1], x_alt_eval[i - rnn_length + 1: i + 1], relation_static = relation_static)
        output = output.detach().cpu()
        preds.append(np.exp(output.numpy()))
        trues.append(y_eval[i].cpu().numpy())
    acc, auc = metrics(trues, preds)
    return acc,  auc

In [5]:

# DEVICE = "cuda:" + device
DEVICE = "cpu"
criterion = torch.nn.NLLLoss()
set_seed(1017)
if relation != "None":
    static = 1
    pass
else:
    static = 0
    relation_static = None
# load dataset
print("loading dataset")
x, y, x_sentiment, relation_static = load_dataset(DEVICE, relation)
# hyper-parameters
NUM_STOCK = x.size(1)
D_MARKET = x.size(2)
D_NEWS = x_sentiment.size(2)
MAX_EPOCH =  max_epoch
infer = infer
hidn_rnn = hidn_rnn
heads_att = heads_att
hidn_att= hidn_att
lr = lr
rnn_length = rnn_length
t_mix = 1
#train-test split
x_train = x[: -140]
x_eval = x[-140 - rnn_length : -70]
x_test = x[-70 - rnn_length:]

y_train = y[: -140]
y_eval = y[-140 - rnn_length : -70]
y_test = y[-70 - rnn_length:]

x_sentiment_train = x_sentiment[: -140]
x_sentiment_eval = x_sentiment[-140 - rnn_length : -70]
x_sentiment_test = x_sentiment[-70 - rnn_length:]
# initialize
best_model_file = 0
epoch = 0
wait_epoch = 0
eval_epoch_best = 0

model = AD_GAT(num_stock=NUM_STOCK, d_market = D_MARKET,d_news= D_NEWS,
                  d_hidden = D_MARKET, hidn_rnn = hidn_rnn, heads_att = heads_att,
                  hidn_att= hidn_att, dropout = dropout,t_mix = t_mix,
                  infer = infer, relation_static = static)
# model.cuda(device=DEVICE)
model.to(torch.double)
optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=weight_constraint)
#train
while epoch < MAX_EPOCH:
    train_loss = train(model, x_train,x_sentiment_train, y_train, relation_static = relation_static)
    eval_acc, eval_auc = evaluate(model, x_eval, x_sentiment_eval, y_eval, relation_static = relation_static)
    test_acc, test_auc = evaluate(model, x_test, x_sentiment_test, y_test, relation_static = relation_static)
    eval_str = "epoch{}, train_loss{:.4f}, eval_auc{:.4f}, eval_acc{:.4f}, test_auc{:.4f},test_acc{:.4f}".format(epoch, train_loss, eval_auc, eval_acc, test_auc, test_acc)
    print(eval_str)

    if eval_auc > eval_epoch_best:
        eval_epoch_best = eval_auc
        eval_best_str = "epoch{}, train_loss{:.4f}, eval_auc{:.4f}, eval_acc{:.4f}, test_auc{:.4f},test_acc{:.4f}".format(epoch, train_loss, eval_auc,eval_acc, test_auc, test_acc)
        wait_epoch = 0
        if save:
            if best_model_file:
                os.remove(best_model_file)
            best_model_file = "./SavedModels/eval:auc{}_acc{}_test:auc{}_acc{}".format(eval_auc, eval_acc, test_auc, test_acc)
            torch.save(model.state_dict(), best_model_file)
    else:
        wait_epoch += 1

    if wait_epoch > 50:
        print("saved_model_result:",eval_best_str)
        break
    epoch += 1


loading dataset


  0%|                                                                       | 0/1339 [00:03<?, ?it/s]


RuntimeError: The size of tensor a (424) must match the size of tensor b (198) at non-singleton dimension 1

In [ ]:
x.shape

In [ ]:
x_sentiment.shape

In [ ]:
y.shape

In [ ]:
relation_static.shape

In [ ]:
x_sentiment_train.shape